In [1]:
import pandas as pd
import random
import pprint

from transformers import AutoModel, AutoTokenizer

from torch import nn
import torch.nn.functional as F

from sklearn.metrics import accuracy_score, f1_score

In [2]:
data = pd.read_excel("data.xlsx", engine="openpyxl")#.iloc[:100]
# data["label_idx"] = [random.choice([0,1,2,3,4]) for _ in range(100)]
display(data.head(2))
data.shape

,sentence,label_idx
0,후기 남길게요 오븐 스파게티 서비스로 주세요,0
1,혹시 코스요리에서 파스타만 포장 가능할까요?,0


(15726, 2)

In [3]:
all_indices = data.index

In [4]:
train_size = int(0.8 * len(all_indices))
val_size = len(all_indices) - train_size
train_idx, val_idx = all_indices[:int(0.8 * len(all_indices))], all_indices[int(0.8 * len(all_indices)):]

num_classes = max(data["label_idx"].tolist())+1

In [5]:
train_idx, val_idx

(RangeIndex(start=0, stop=12580, step=1),
 RangeIndex(start=12580, stop=15726, step=1))

In [6]:
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import BertForSequenceClassification, BertTokenizer, AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm.auto import tqdm

# transformers 라이브러리에서 BERT 토크나이저를 임포트합니다.
from transformers import BertTokenizer

# 다국어 지원 BERT 모델을 사용하여 토크나이저 객체를 생성합니다.
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-small")

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 데이터셋 클래스 정의
class TextClassificationDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.encodings = [
            tokenizer.encode_plus(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512) for sentence in df["sentence"].tolist()
        ]
        self.labels = df["label_idx"].tolist()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # to_return = {
        #     "sentence_encoded":self.encodings[idx],
        #     "label":self.labels[idx]
        # }
        return (self.encodings[idx], self.labels[idx])
        # return to_return

In [31]:
class TextClassificationDatasetBatch(Dataset):
    def __init__(self, df, tokenizer, batch_size=4):
        self.tokenizer = tokenizer
        self.sentences = df["sentence"].tolist()
        self.labels = df["label_idx"].tolist()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # to_return = {
        #     "sentence_encoded":self.encodings[idx],
        #     "label":self.labels[idx]
        # }
        return (self.sentences[idx], self.labels[idx])

In [32]:
train_data = data.iloc[train_idx]
val_data = data.iloc[val_idx]

In [35]:
# train_dataset = TextClassificationDataset(train_data, tokenizer)
# val_dataset = TextClassificationDataset(val_data, tokenizer)
train_dataset = TextClassificationDatasetBatch(train_data, tokenizer)
val_dataset = TextClassificationDatasetBatch(val_data, tokenizer)

In [36]:
train_dataset[0]

('후기 남길게요 오븐 스파게티 서비스로 주세요', 0)

In [45]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)

In [46]:
t = next(iter(train_dataloader))

In [47]:
t[0]

('양갈비는 잘 안 나가요?',
 '국밥 굴국밥 이런거는 어떠세요?',
 '중복 할인은 안돼요',
 '미디움레어로 해주세요',
 '네 잠시만 기다리세요',
 '영수증에 대기번호도 같이 써있습니다',
 '네',
 '우리은행도 있고 현대카드도 있는데 그거는 할인 없고?',
 '이 동네 사세요?',
 '예 정식은 가격이 얼마에요?')

In [48]:
t[1]

tensor([2806, 1101, 1443, 2236,    0,  747,    0, 1180, 2759, 1037])

In [148]:
def collate_bert_tokernizer(samples):
    # print("samples :: ", samples)
    sentences, lables = [s[0] for s in samples], [s[1] for s in samples]
    global tokenizer
    return tokenizer(sentences, return_tensors="pt", padding=True, truncation=True, max_length=512), lables

In [149]:
train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True, collate_fn = collate_bert_tokernizer) #

In [132]:
# t = next(iter(train_dataloader))

In [133]:
# model(**train_dataset[0][0])["pooler_output"].shape

In [134]:
# tokenized = tokenizer(["이것은 문장입니다"], return_tensors="pt", padding=True, truncation=True, max_length=512)
# print(tokenized["input_ids"].shape)
# model(**tokenized)["pooler_output"].shape

In [164]:
tokenized = tokenizer(["이것은 문장입니다"] * 4, return_tensors="pt", padding=True, truncation=True, max_length=512)
print(tokenized["input_ids"].shape)
model(**tokenized)["pooler_output"].shape

torch.Size([4, 6])


torch.Size([4, 768])

In [175]:
tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [174]:
model(input_ids=tokenized["input_ids"], token_type_ids=tokenized["token_type_ids"], attention_mask=tokenized["attention_mask"])

{'input_ids': tensor([[    0,  3982,  2073,  6265, 12190,     2],
        [    0,  3982,  2073,  6265, 12190,     2],
        [    0,  3982,  2073,  6265, 12190,     2],
        [    0,  3982,  2073,  6265, 12190,     2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1]])}

In [ ]:
model(**input_ids)

In [136]:
# batch_size = 4
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [165]:
model = AutoModel.from_pretrained("klue/roberta-small")

class CustomModel(nn.Module):
    def __init__(self, bert_model, num_classes, hidden_size):
        super(CustomModel, self).__init__()
        self.model = bert_model
        self.num_classes = num_classes
        
        self.fc = nn.Linear(hidden_size, num_classes) 
    
    def forward(self, _input):
        output = self.model(**_input)["pooler_output"]
        output = self.fc(output)
        return output

custom_model = CustomModel(model, num_classes, 768)
optimizer = AdamW(custom_model.parameters(), lr=0.001)

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/dwook.kim/miniconda3/envs/py38/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [166]:
for item in train_dataloader:
    break

In [173]:
torch.Tensor(item[1], dtype=torch.long)

TypeError: new() received an invalid combination of arguments - got (list, dtype=torch.dtype), but expected one of:
 * (*, torch.device device)
      didn't match because some of the keywords were incorrect: dtype
 * (torch.Storage storage)
 * (Tensor other)
 * (tuple of ints size, *, torch.device device)
 * (object data, *, torch.device device)


In [168]:
loss_f = nn.CrossEntropyLoss()
epochs = 2

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    print('-' * 10)
    # 모델을 학습 모드로 설정합니다.
    model.train()
    for i, (_input, _label) in enumerate(train_dataloader):
        _label = torch.tensor(F.one_hot(torch.tensor(_label, dtype=torch.long), num_classes=num_classes), dtype=torch.float).squeeze()
        
        outputs = custom_model(_input)
        loss = loss_f(F.softmax(outputs), _label)
        loss.backward() # 역전파를 수행합니다.
        optimizer.step()
        optimizer.zero_grad()
        break
    print(outputs)
    model.eval()
    all_predictions = []
    all_labels = []

    # for i, (_input, _label) in enumerate(val_loader):
    #     _input = {k:v.squeeze(0) for k, v in _input.items()}
    #     _label = torch.tensor(F.one_hot(_label, num_classes=num_classes), dtype=torch.float).squeeze()
        
    #     outputs = custom_model(_input)
    #     #loss = loss_f(F.softmax(outputs), _label)
    #     all_predictions.append( torch.argmax(F.softmax(outputs)).item() )
    #     all_labels.append( torch.argmax(F.softmax(_label)).item() )
        


    # print(accuracy_score(all_labels, all_predictions))
    # print(f1_score(all_labels, all_predictions, average="weighted"))
    

Epoch 1/2
----------


/var/folders/3g/y74d9tn12ls8tx3g7nmyjc3h0000gn/T/ipykernel_7454/4120136601.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _label = torch.tensor(F.one_hot(torch.tensor(_label, dtype=torch.long), num_classes=num_classes), dtype=torch.float).squeeze()
/var/folders/3g/y74d9tn12ls8tx3g7nmyjc3h0000gn/T/ipykernel_7454/4120136601.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  loss = loss_f(F.softmax(outputs), _label)


tensor([[ 0.0174, -0.1574,  0.2068,  ...,  0.1841, -0.0257,  0.2325],
        [-0.0897, -0.1325,  0.1657,  ...,  0.0426, -0.0107,  0.0971],
        [-0.0063, -0.1700,  0.0681,  ...,  0.0938, -0.0737,  0.1298],
        ...,
        [-0.0067, -0.1154,  0.1552,  ...,  0.1138, -0.0716,  0.1311],
        [-0.0688, -0.1137,  0.1341,  ...,  0.0941, -0.0464,  0.1196],
        [-0.1594, -0.0395,  0.0297,  ...,  0.0951, -0.0749,  0.0596]],
       grad_fn=<AddmmBackward0>)
Epoch 2/2
----------
tensor([[ 0.3736, -0.0962,  0.2207,  ...,  0.1817, -0.0201,  0.2081],
        [ 0.4112, -0.1317,  0.2172,  ...,  0.1246, -0.0584,  0.2213],
        [ 0.3402, -0.2247,  0.2087,  ...,  0.1722, -0.0382,  0.1058],
        ...,
        [ 0.3244, -0.1503,  0.1475,  ...,  0.1443, -0.1613,  0.1717],
        [ 0.3556, -0.1773,  0.1659,  ...,  0.1727, -0.0050,  0.2692],
        [ 0.4056, -0.0415,  0.2137,  ...,  0.1232, -0.2285,  0.0653]],
       grad_fn=<AddmmBackward0>)


In [170]:
outputs[0]

tensor([ 0.3736, -0.0962,  0.2207,  ...,  0.1817, -0.0201,  0.2081],
       grad_fn=<SelectBackward0>)

In [171]:
outputs[1]

tensor([ 0.4112, -0.1317,  0.2172,  ...,  0.1246, -0.0584,  0.2213],
       grad_fn=<SelectBackward0>)